#TODO

El algoritmo consiste principalmente en un loop que para cada archivo:
1) Leer, para tenerlo en un dataframe (OK)
2) Filtrar, para quedarnos con los registros y columnas relevantes (OK)
3) Agregar, datos adicionales, para poder trabajar mejor:
    1) Agregar mes y año del archivo, para usar en lugar de la fecha de creación (OK)
    2) Agregar todos los datos que se podrían obtener del campo descripición, como cochera, vista al mar, etc (TODO)
    3) Agregar todos los datos adicionales resultantes de la unión con los otros dataframes (hospitales, subte, etc) (TODO)
4) Finalmente, para los registros que no tengan precio, calcular el valor: (TODO)
    1) Obtener las propiedades más semejantes (como con KNN):
        1) Restricción: La distancia temporal entre propiedades no puede ser mayor a 3-6 meses.
    2) En base a eso, obtener el coeficiente promedio (valor propiedad / (precio unitario promedio * superficie total))
    3) Finalmente, multiplicar el coeficiente por precio unitario promedio y superficie
5) Agregar el dataframe a uno global, para poder predecir mejor las siguientes propiedades, en cuanto a la distancia temporal (TODO)

Finalmente, se realiza el reporte de las propiedades de las que se quiere obtener el valor (en este caso, agosto 2017) (TODO)

In [ ]:
# No vamos a graficar, de momento
#%matplotlib inline

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from os import listdir

In [ ]:
#Funciones de utilería

def SeleccionarSubCampo(campo, separador, indice):
    """Selecciona un elemento de una lista en string"""
    return campo.split(separador)[indice]

def SacarColumna(dataFrame, columnName):
    """Quita una columna de un dataframe. Si no existe, no hace nada"""
    if columnName in dataFrame:
        dataFrame.drop(columnName, axis = 1, inplace = True)

def SacarListaColumnas(dataFrame, columnNameList):
    """Quita una lista de columnas de un dataframe"""
    for columnName in columnNameList:
        SacarColumna(dataFrame, columnName)

def ResolveColumn(dataFrame, columnNameFrom, columnNameTo, function):
    dataFrame[columnNameTo] = pd.Series([function(x) for x in dataFrame[columnNameFrom]], index = dataFrame.index)

In [ ]:
def LeerDataFrame(nombreArchivo):
    """Función que lee un archivo y devuelve un dataframe"""
    return pd.read_csv(nombreArchivo, low_memory = False)

# Funciones que filtran datos
def FiltrarPais(dataFrame, listaPaises):
    """Devuelve un dataframe con la lista de países filtrado"""
    if not 'country' in dataFrame:
        ResolveColumn(dataFrame, 'place_with_parent_names', 'country', lambda x: SeleccionarSubCampo(x, '|', 1))
    dataFrame = dataFrame[dataFrame['country'].isin(listaPaises)]
    SacarColumna(dataFrame, 'country')
    return dataFrame

def FiltrarProvincia(dataFrame, listaProvincias):
    """Devuelve un dataframe con la lista de países filtrado"""
    if not 'state_name' in dataFrame:
        ResolveColumn(dataFrame, 'place_with_parent_names', 'state_name', lambda x: SeleccionarSubCampo(x, '|', 2))
    dataFrame = dataFrame[dataFrame['state_name'].isin(listaProvincias)]
    return dataFrame

def EliminarColumnasNoRelevantes(dataSet):
    SacarListaColumnas(dataSet, ['properati_url',
                                 'operation',
                                 'geonames_id',
                                 'lat-lon',
                                 'currency',
                                 'price_aprox_local_currency',
                                 'image_thumbnail'])

def AplicarFiltros(dataFrame):
    dataFrame = FiltrarPais(dataFrame, ['Argentina'])
    dataFrame = FiltrarProvincia(dataFrame, ['Capital Federal',
                               'Bs.As. G.B.A. Zona Norte',
                               'Bs.As. G.B.A. Zona Oeste',
                               'Bs.As. G.B.A. Zona Sur'])
    EliminarColumnasNoRelevantes(dataFrame)
    dataFrame.reindex()
    return dataFrame

# Funciones que agregan información
def AgregarFechaDeArchivo(dataFrame, archivo):
    archiveParts = archivo.split('-')
    dumpDate = archiveParts[2] + '-' + archiveParts[3]
    dataFrame['dump_date'] = pd.Series(dumpDate, index = dataFrame.index)

def AgregarBarrio(dataFrame):
    if not 'barrio' in dataFrame:
        ResolveColumn(dataFrame, 'place_with_parent_names', 'barrio', lambda x: SeleccionarSubCampo(x, '|', 3))
    SacarColumna(dataFrame, 'place_with_parent_names')

def AgregarInformacion(dataFrame, archivo):
    AgregarFechaDeArchivo(dataFrame, archivo)
    AgregarBarrio(dataFrame)
    return dataFrame

In [ ]:
#Funciones útiles

def CrearDataFrame(columnas):
    """Crea un dataframe vacío, cuyas columnas se crean a
    partir de una lista de pares nombre y dtype"""
    dataframe = pd.DataFrame()
    for col in columnas_gBarriosPrecios:
        dataframe[col[0]] = pd.Series(name = col[0], dtype = col[1])
    return dataframe


In [ ]:
#variables "globales"

# Ruta de la carpeta con los archivos de datos originales
gRutaCarpetaOriginales = "./properties/"

#TODO: revisar si vale la pena
# Dataframe con estadísticas de precio por barrio y mes

columnas_gBarriosPrecios = [('state_name', np.dtype('str')),
                            ('place_name', np.dtype('str')),
                            ('entry_date', np.dtype('str')),
                            ('mean price', np.dtype('float')),
                            ('min_price', np.dtype('float')),
                            ('max_price', np.dtype('float'))]

gBarriosPrecios = CrearDataFrame(columnas_gBarriosPrecios)

gBarriosPrecios.info()

In [ ]:
# Hacemos un rápido chequeo de los archivos, para saber si alguno se descargó mal
def CheckCorruptDownloads():
    for archive in listdir(gRutaCarpetaOriginales):
        if ".csv" in archive:
            nombreArchivo = gRutaCarpetaOriginales + archive
            try:
                print(nombreArchivo, end = ': ')
                df = pd.read_csv(nombreArchivo, low_memory = False)
                print('Ok')
            except ValueError:
                print("Error, archivo corrupto. Descargar de nuevo")

#CheckCorruptDownloads()

In [ ]:
def ProcesarDataFrame(rutaArchivo, nombreArchivo):
    """Procesa un archivo csv, y devuelve un dataFrame listo para usar"""
    dataFrame = LeerDataFrame(rutaArchivo)
    dataFrame = AplicarFiltros(dataFrame)
    dataFrame = AgregarInformacion(dataFrame, nombreArchivo)
    return dataFrame

# loop principal
for archive in listdir(gRutaCarpetaOriginales):
    if ".csv" in archive:
        print(archive)
        df = ProcesarDataFrame(gRutaCarpetaOriginales + archive, archive)

        if 'price' not in df:
            df['price'] = pd.Series(name = 'price', dtype = np.dtype('float'))

        #TODO: Calcular precio en caso de nulo o cero

        print(df.info())
    break